
## Módulos de Inserção

Os módulos de inserção, ou *Insertion Modules*, são uma arquitetura de redes neurais convolucionais (CNNs) projetada para capturar características em diferentes escalas dentro de uma imagem. Vamos definir o módulo `InsercaoModulo`, que realiza as seguintes operações:

1. **Redução de Dimensão**: Utiliza uma convolução 1x1 para reduzir o número de canais de entrada, facilitando a integração de diferentes características.
2. **Convoluções**:
   - **Convolução 1x1**: Captura características de baixo nível mantendo a dimensionalidade dos canais.
   - **Convolução 3x3**: Extrai características espaciais mais detalhadas.
   - **Convolução 5x5**: Captura padrões mais amplos e complexos.
3. **Max Pooling**: Aplica uma operação de max pooling 3x3 para reduzir a resolução espacial, seguida por uma convolução 1x1 para manter a dimensionalidade dos canais.
4. **Concatenação**: Combina as saídas das operações anteriores ao longo da dimensão dos canais, integrando informações de diferentes escalas.


### Importando os módulos

In [1]:

import torch
import torch.nn as nn


#### Criando o modelo de módulo de inserção com redução de dimensões

In [2]:

class InsercaoModulo(nn.Module):
    def __init__(self, in_channels=3, reduced_channels=2, out_channels=32):
        super(InsercaoModulo, self).__init__()

        # Redução de Dimensão com Convolução 1x1
        self.reduction = nn.Conv2d(in_channels, reduced_channels, kernel_size=1)

        # Convolução 1x1 após redução
        self.conv1x1 = nn.Conv2d(reduced_channels, out_channels, kernel_size=1)

        # Convolução 3x3 após redução
        self.conv3x3 = nn.Conv2d(reduced_channels, out_channels, kernel_size=3, padding=1)

        # Convolução 5x5 após redução
        self.conv5x5 = nn.Conv2d(reduced_channels, out_channels, kernel_size=5, padding=2)

        # Max Pooling 3x3 seguido de convolução 1x1 após redução
        self.maxpool3x3 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_after_pooling = nn.Conv2d(reduced_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # Aplicando a redução de dimensão
        x_reduced = self.reduction(x)

        # Aplicando cada uma das operações após a redução
        conv1x1_out = self.conv1x1(x_reduced)
        conv3x3_out = self.conv3x3(x_reduced)
        conv5x5_out = self.conv5x5(x_reduced)
        maxpool_out = self.conv_after_pooling(self.maxpool3x3(x_reduced))

        # Concatenando as saídas ao longo da dimensão dos canais
        out = torch.cat([conv1x1_out, conv3x3_out, conv5x5_out, maxpool_out], dim=1)

        return out


### Exemplo de Utilização

Vamos considerar um exemplo prático de como utilizar o módulo `InsercaoModulo`:

1. **Número de Canais de Entrada**: 3 canais (por exemplo, uma imagem RGB).
2. **Número de Canais Após Redução**: 2 canais, obtidos após a operação de redução de dimensão com uma convolução 1x1.
3. **Número de Canais de Saída**: 32 canais para cada operação de convolução (1x1, 3x3 e 5x5) e após a operação de max pooling com convolução adicional.


### Teste

---



Vamos agora testar o módulo `InsercaoModulo` com um tensor de entrada que simula uma imagem RGB (com 3 canais). Vamos observar a forma da saída após passar pelo módulo.


In [3]:
in_channels = 3  # Número de canais de entrada (3 para uma imagem RGB)
reduced_channels = 2  # Número de canais após a redução
out_channels = 32  # Número de canais de saída por cada operação de convolução

model = InsercaoModulo(in_channels, reduced_channels, out_channels)
input_tensor = torch.randn(1, in_channels, 32, 32)  # Um tensor de entrada de exemplo (batch size 1, 3 canais, 32x32 de dimensão)
output = model(input_tensor)

O valor de saída é:

In [ ]:
print(output.shape)

torch.Size([3, 128, 32, 32])


 ## Classificadores Auxiliares


Auxiliary classifiers, ou classificadores auxiliares, são componentes adicionais em uma rede neural projetada para ajudar a melhorar o desempenho do modelo principal. Eles são usados para realizar tarefas secundárias que estão relacionadas à tarefa principal, mas não são a tarefa principal em si. Esses classificadores podem ter várias finalidades, como melhorar a qualidade das representações aprendidas, fornecer regularização ou incentivar a rede a aprender características mais robustas.

### Importando os módulos.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

 ### Criando um exemplo de rede neural que possui um classificador auxiliar.

In [2]:


# Definindo a Rede Principal com Classificador Auxiliar
class DeepNetworkWithAuxiliary(nn.Module):
    def __init__(self, num_classes=10):
        super(DeepNetworkWithAuxiliary, self).__init__()

        # Camadas iniciais da rede
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Camada intermediária onde aplicaremos o classificador auxiliar
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Classificador Auxiliar
        self.aux_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, num_classes)
        )

        # Camadas finais da rede
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        # Passando pela camada intermediária
        x = torch.relu(self.conv3(x))

        # Extraindo a saída do classificador auxiliar
        aux_output = self.aux_classifier(x)

        # Continuando na rede principal
        x = torch.relu(self.conv4(x))
        x = F.adaptive_avg_pool2d(x, (1, 1))  # Usando a função do módulo functional
        x = torch.flatten(x, 1)
        main_output = self.fc(x)

        return main_output, aux_output



### Criando a rede.

In [3]:
# Criando a rede e definindo a função de perda e otimizador
model = DeepNetworkWithAuxiliary(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Exemplo de uso

 Utilizando um batch aleatório de 16 imagens RGB 32x32 e 16 rótulos de classe:

In [4]:

input_tensor = torch.randn(16, 3, 32, 32)
labels = torch.randint(0, 10, (16,))

## Treino

Utilizando o valor  0.4 para a perda auxiliar, vamos realizar o treino.

In [5]:

optimizer.zero_grad()
main_output, aux_output = model(input_tensor)
loss_main = criterion(main_output, labels)
loss_aux = criterion(aux_output, labels)
total_loss = loss_main + 0.4 * loss_aux
total_loss.backward()
optimizer.step()
print("saída principal do modelo:", main_output)
print('')
print("saída auxiliar do modelo:",aux_output)
print('')
print("Perda total:", total_loss.item())
print("Perda Auxiliar:",loss_aux.item())

saída principal do modelo: tensor([[-0.0340, -0.0190, -0.0407, -0.0161, -0.0114,  0.0136,  0.0263,  0.0038,
         -0.0248, -0.0262],
        [-0.0344, -0.0194, -0.0406, -0.0155, -0.0113,  0.0139,  0.0265,  0.0037,
         -0.0248, -0.0260],
        [-0.0344, -0.0197, -0.0403, -0.0149, -0.0117,  0.0135,  0.0267,  0.0044,
         -0.0244, -0.0259],
        [-0.0345, -0.0189, -0.0406, -0.0155, -0.0117,  0.0134,  0.0268,  0.0040,
         -0.0243, -0.0260],
        [-0.0346, -0.0194, -0.0405, -0.0158, -0.0112,  0.0135,  0.0265,  0.0041,
         -0.0244, -0.0259],
        [-0.0341, -0.0190, -0.0410, -0.0153, -0.0110,  0.0134,  0.0266,  0.0036,
         -0.0243, -0.0263],
        [-0.0344, -0.0188, -0.0411, -0.0156, -0.0110,  0.0131,  0.0267,  0.0037,
         -0.0245, -0.0261],
        [-0.0344, -0.0189, -0.0407, -0.0156, -0.0117,  0.0127,  0.0265,  0.0042,
         -0.0240, -0.0261],
        [-0.0345, -0.0190, -0.0405, -0.0153, -0.0115,  0.0134,  0.0265,  0.0042,
         -0.0244, -0

### Calcular a acurácia do modelo

In [6]:
_, predicted = torch.max(main_output, 1)
correct = (predicted == labels).sum().item()
total = labels.size(0)
accuracy = correct / total
print("Precisão:", accuracy)

Precisão: 0.0625
